In [1]:
import pandas as pd
import os
import sys

In [36]:
import fbprophet as fb

In [2]:
file_path = 'C:\\Users\\Rohan\\Desktop\\Kaggle\\M5_Walmart\\'

In [3]:
df_calendar = pd.read_csv(file_path + 'calendar.csv',parse_dates=[0])

In [9]:
df_calendar[['unwanted','date_sk']] = df_calendar.date_key.str.split("_",expand=True)
df_calendar.drop(columns=['unwanted'],inplace=True)
df_calendar.year = df_calendar.year.astype('int64')
df_calendar.date_sk = df_calendar.date_sk.astype('int64')

In [10]:
df_calendar.dtypes

date            datetime64[ns]
wm_yr_wk                 int64
weekday                 object
wday                     int64
month                    int64
year                     int64
date_key                object
event_name_1            object
event_type_1            object
event_name_2            object
event_type_2            object
snap_CA                  int64
snap_TX                  int64
snap_WI                  int64
date_sk                  int64
dtype: object

In [4]:
df_read_full_file = pd.read_csv(file_path + 'sales_train_validation_normalized.csv')

In [5]:
df_read_full_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   id             object
 1   item_sk        int64 
 2   store_sk       int64 
 3   date           object
 4   unit_sold      int64 
 5   trasaction_id  int64 
dtypes: int64(4), object(2)
memory usage: 2.6+ GB


In [6]:
df_read_full_file.head()

,id,item_sk,store_sk,date,unit_sold,trasaction_id
0,HOBBIES_1_001_CA_1_validation,1,1,d_1,0,1
1,HOBBIES_1_001_CA_1_validation,1,1,d_2,0,2
2,HOBBIES_1_001_CA_1_validation,1,1,d_3,0,3
3,HOBBIES_1_001_CA_1_validation,1,1,d_4,0,4
4,HOBBIES_1_001_CA_1_validation,1,1,d_5,0,5


In [21]:
df_item_sk_1 = df_read_full_file[(df_read_full_file.item_sk == 1) & (df_read_full_file.store_sk == 1)]

In [8]:
df_item_sk_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1913 entries, 0 to 1912
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1913 non-null   object
 1   item_sk        1913 non-null   int64 
 2   store_sk       1913 non-null   int64 
 3   date           1913 non-null   object
 4   unit_sold      1913 non-null   int64 
 5   trasaction_id  1913 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 104.6+ KB


In [25]:
df_item_sk_1[['unwanted','date_sk2']] = df_item_sk_1.date.str.split("_",expand=True)

C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [32]:
df_item_sk_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1913 entries, 0 to 1912
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1913 non-null   object
 1   item_sk        1913 non-null   int64 
 2   store_sk       1913 non-null   int64 
 3   unit_sold      1913 non-null   int64 
 4   trasaction_id  1913 non-null   int64 
 5   date_sk        1913 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 104.6+ KB


In [28]:
df_item_sk_1.drop(columns=['unwanted','date'],inplace=True)
df_item_sk_1.rename(columns={'date_sk2':'date_sk'},inplace=True)

C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
df_item_sk_1.head()

,id,item_sk,store_sk,unit_sold,trasaction_id,date_sk
0,HOBBIES_1_001_CA_1_validation,1,1,0,1,1
1,HOBBIES_1_001_CA_1_validation,1,1,0,2,2
2,HOBBIES_1_001_CA_1_validation,1,1,0,3,3
3,HOBBIES_1_001_CA_1_validation,1,1,0,4,4
4,HOBBIES_1_001_CA_1_validation,1,1,0,5,5


In [31]:
df_item_sk_1.date_sk = df_item_sk_1.date_sk.astype('int64')

C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [33]:
df_item_sk_1 = df_item_sk_1.join(df_calendar.set_index('date_sk'),on=['date_sk'],rsuffix='xxx')

In [35]:
df_item_sk_1.groupby(by=['year']).count()

,id,item_sk,store_sk,unit_sold,trasaction_id,date_sk,date,wm_yr_wk,weekday,wday,month,date_key,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
year,,,,,,,,,,,,,,,,,,,
2011,337,337,337,337,337,337,337,337,337,337,337,337,26,26,1,1,337,337,337
2012,366,366,366,366,366,366,366,366,366,366,366,366,30,30,0,0,366,366,366
2013,365,365,365,365,365,365,365,365,365,365,365,365,29,29,1,1,365,365,365
2014,365,365,365,365,365,365,365,365,365,365,365,365,28,28,2,2,365,365,365
2015,365,365,365,365,365,365,365,365,365,365,365,365,30,30,0,0,365,365,365
2016,115,115,115,115,115,115,115,115,115,115,115,115,11,11,0,0,115,115,115


In [49]:
df1_for_fb = df_item_sk_1[['date','unit_sold']]

In [50]:
df1_for_fb.head()

,date,unit_sold
0,2011-01-29,0
1,2011-01-30,0
2,2011-01-31,0
3,2011-02-01,0
4,2011-02-02,0


In [51]:
df1_for_fb.rename(columns={'date':'ds','unit_sold':'y'},inplace=True)

C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [52]:
df1_for_fb

,ds,y
0,2011-01-29,0
1,2011-01-30,0
2,2011-01-31,0
3,2011-02-01,0
4,2011-02-02,0
...,...,...
1908,2016-04-20,1
1909,2016-04-21,3
1910,2016-04-22,0
1911,2016-04-23,1


In [53]:
m = fb.Prophet()

In [55]:
m.fit(df1_for_fb)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [71]:
future = m.make_future_dataframe(periods=30)

In [72]:
forecast = m.predict(future)

In [73]:
forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1943 entries, 0 to 1942
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ds                          1943 non-null   datetime64[ns]
 1   trend                       1943 non-null   float64       
 2   yhat_lower                  1943 non-null   float64       
 3   yhat_upper                  1943 non-null   float64       
 4   trend_lower                 1943 non-null   float64       
 5   trend_upper                 1943 non-null   float64       
 6   additive_terms              1943 non-null   float64       
 7   additive_terms_lower        1943 non-null   float64       
 8   additive_terms_upper        1943 non-null   float64       
 9   weekly                      1943 non-null   float64       
 10  weekly_lower                1943 non-null   float64       
 11  weekly_upper                1943 non-null   float64     

In [60]:
forecast.ds.min()

Timestamp('2011-01-29 00:00:00')

In [74]:
forecast.ds.max()

Timestamp('2016-05-24 00:00:00')

In [62]:
df_calendar.date.min()

Timestamp('2011-01-29 00:00:00')

In [77]:
forecast = forecast.join(df_calendar.set_index('date'),on=['ds'],rsuffix='xxx')

In [67]:
forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278 entries, 0 to 2277
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ds                          2278 non-null   datetime64[ns]
 1   trend                       2278 non-null   float64       
 2   yhat_lower                  2278 non-null   float64       
 3   yhat_upper                  2278 non-null   float64       
 4   trend_lower                 2278 non-null   float64       
 5   trend_upper                 2278 non-null   float64       
 6   additive_terms              2278 non-null   float64       
 7   additive_terms_lower        2278 non-null   float64       
 8   additive_terms_upper        2278 non-null   float64       
 9   weekly                      2278 non-null   float64       
 10  weekly_lower                2278 non-null   float64       
 11  weekly_upper                2278 non-null   float64     

In [78]:
forecast.groupby(by=['year']).count()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,...,month,date_key,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,date_sk
year,,,,,,,,,,,,,,,,,,,,,
2011,337,337,337,337,337,337,337,337,337,337,...,337,337,26,26,1,1,337,337,337,337
2012,366,366,366,366,366,366,366,366,366,366,...,366,366,30,30,0,0,366,366,366,366
2013,365,365,365,365,365,365,365,365,365,365,...,365,365,29,29,1,1,365,365,365,365
2014,365,365,365,365,365,365,365,365,365,365,...,365,365,28,28,2,2,365,365,365,365
2015,365,365,365,365,365,365,365,365,365,365,...,365,365,30,30,0,0,365,365,365,365
2016,145,145,145,145,145,145,145,145,145,145,...,145,145,15,15,0,0,145,145,145,145


In [70]:
df_calendar.date.max()

Timestamp('2016-06-19 00:00:00')

In [79]:
forecast[['ds','yhat']][forecast.ds > '2016-04-24']

,ds,yhat
1913,2016-04-25,0.762079
1914,2016-04-26,0.822200
1915,2016-04-27,0.706412
1916,2016-04-28,0.696686
1917,2016-04-29,0.749047
1918,2016-04-30,0.896207
1919,2016-05-01,0.772794
1920,2016-05-02,0.721267
1921,2016-05-03,0.780071
1922,2016-05-04,0.662868


In [80]:
forecast['item_sk'] = 1

In [81]:
forecast['store_sk'] = 1

In [85]:
forecast_item_sk1 = forecast[['item_sk','store_sk','ds','date_key','yhat']][forecast.ds > '2016-04-24']

In [86]:
forecast_item_sk1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 1913 to 1942
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   item_sk   30 non-null     int64         
 1   store_sk  30 non-null     int64         
 2   ds        30 non-null     datetime64[ns]
 3   date_key  30 non-null     object        
 4   yhat      30 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.4+ KB


In [88]:
forecast_item_sk1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 1913 to 1942
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   item_sk   30 non-null     int64         
 1   store_sk  30 non-null     int64         
 2   ds        30 non-null     datetime64[ns]
 3   date_key  30 non-null     object        
 4   yhat      30 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.4+ KB


In [89]:
df_read_full_file.groupby(by=['item_sk','store_sk']).count()

id  date  unit_sold  trasaction_id
item_sk store_sk                                      
1       1         1913  1913       1913           1913
        2         1913  1913       1913           1913
        3         1913  1913       1913           1913
        4         1913  1913       1913           1913
        5         1913  1913       1913           1913
...                ...   ...        ...            ...
3049    6         1913  1913       1913           1913
        7         1913  1913       1913           1913
        8         1913  1913       1913           1913
        9         1913  1913       1913           1913
        10        1913  1913       1913           1913

[30490 rows x 4 columns]

In [92]:
df_item_store_combo = df_read_full_file[['item_sk','store_sk']][df_read_full_file.duplicated(subset=['item_sk','store_sk']) == False]

In [95]:
df_item_store_combo = df_item_store_combo.reset_index()

In [96]:
df_item_store_combo

,index,item_sk,store_sk
0,0,1,1
1,1913,2,1
2,3826,3,1
3,5739,4,1
4,7652,5,1
...,...,...,...
30485,58317805,3045,10
30486,58319718,3046,10
30487,58321631,3047,10
30488,58323544,3048,10


In [103]:
for index,i in df_item_store_combo.iterrows():
    print(i['item_sk'],i['store_sk'])
    df_item_sk_1 = df_read_full_file[(df_read_full_file.item_sk == i['item_sk']) & (df_read_full_file.store_sk == i['store_sk'])]
    df_item_sk_1[['unwanted','date_sk2']] = df_item_sk_1.date.str.split("_",expand=True)
    df_item_sk_1.drop(columns=['unwanted','date'],inplace=True)
    df_item_sk_1.rename(columns={'date_sk2':'date_sk'},inplace=True)
    df_item_sk_1.date_sk = df_item_sk_1.date_sk.astype('int64')
    df_item_sk_1 = df_item_sk_1.join(df_calendar.set_index('date_sk'),on=['date_sk'],rsuffix='xxx')
    df1_for_fb = df_item_sk_1[['date','unit_sold']]
    df1_for_fb.rename(columns={'date':'ds','unit_sold':'y'},inplace=True)
    m = fb.Prophet()
    m.fit(df1_for_fb)
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)

1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 1
44 1
45 1
46 1
47 1
48 1
49 1
50 1
51 1
52 1
53 1
54 1
55 1
56 1
57 1
58 1
59 1
60 1
61 1
62 1
63 1
64 1
65 1
66 1
67 1
68 1
69 1
70 1
71 1
72 1
73 1
74 1
75 1
76 1
77 1
78 1
79 1
80 1
81 1
82 1
83 1
84 1
85 1
86 1
87 1
88 1
89 1
90 1
91 1
92 1
93 1
94 1
95 1
96 1
97 1
98 1
99 1
100 1
101 1
102 1
103 1
104 1
105 1
106 1
107 1
108 1
109 1
110 1
111 1
112 1
113 1
114 1
115 1
116 1
117 1
118 1
119 1
120 1
121 1
122 1
123 1
124 1
125 1
126 1
127 1
128 1
129 1
130 1
131 1
132 1
133 1
134 1
135 1
136 1
137 1
138 1
139 1
140 1
141 1
142 1
143 1
144 1
145 1
146 1
147 1
148 1
149 1
150 1
151 1
152 1
153 1
154 1
155 1
156 1
157 1
158 1
159 1
160 1
161 1
162 1
163 1
164 1
165 1
166 1
167 1
168 1
169 1
170 1
171 1
172 1
173 1
174 1
175 1
176 1
177 1
178 1
179 1
180 1
181 1
182 1
183 1
184 1
185 

1462 1
1463 1
1464 1
1465 1
1466 1
1467 1
1468 1
1469 1
1470 1
1471 1
1472 1
1473 1
1474 1
1475 1
1476 1
1477 1
1478 1
1479 1
1480 1
1481 1
1482 1
1483 1
1484 1
1485 1
1486 1
1487 1
1488 1
1489 1
1490 1
1491 1
1492 1
1493 1
1494 1
1495 1
1496 1
1497 1
1498 1
1499 1
1500 1
1501 1
1502 1
1503 1
1504 1
1505 1
1506 1
1507 1
1508 1
1509 1
1510 1
1511 1
1512 1
1513 1
1514 1
1515 1
1516 1
1517 1
1518 1
1519 1
1520 1
1521 1
1522 1
1523 1
1524 1
1525 1
1526 1
1527 1
1528 1
1529 1
1530 1
1531 1
1532 1
1533 1
1534 1
1535 1
1536 1
1537 1
1538 1
1539 1
1540 1
1541 1
1542 1
1543 1
1544 1
1545 1
1546 1
1547 1
1548 1
1549 1
1550 1
1551 1
1552 1
1553 1
1554 1
1555 1
1556 1
1557 1
1558 1
1559 1
1560 1
1561 1
1562 1
1563 1
1564 1
1565 1
1566 1
1567 1
1568 1
1569 1
1570 1
1571 1
1572 1
1573 1
1574 1
1575 1
1576 1
1577 1
1578 1
1579 1
1580 1
1581 1
1582 1
1583 1
1584 1
1585 1
1586 1
1587 1
1588 1
1589 1
1590 1
1591 1
1592 1
1593 1
1594 1
1595 1
1596 1
1597 1
1598 1
1599 1
1600 1
1601 1
1602 1
1603 1
1604 1

2838 1
2839 1
2840 1
2841 1
2842 1
2843 1
2844 1
2845 1
2846 1
2847 1
2848 1
2849 1
2850 1
2851 1
2852 1
2853 1
2854 1
2855 1
2856 1
2857 1
2858 1
2859 1
2860 1
2861 1
2862 1
2863 1
2864 1
2865 1
2866 1
2867 1
2868 1
2869 1
2870 1
2871 1
2872 1
2873 1
2874 1
2875 1
2876 1
2877 1
2878 1
2879 1
2880 1
2881 1
2882 1
2883 1
2884 1
2885 1
2886 1
2887 1
2888 1
2889 1
2890 1
2891 1
2892 1
2893 1
2894 1
2895 1
2896 1
2897 1
2898 1
2899 1
2900 1
2901 1
2902 1
2903 1
2904 1
2905 1
2906 1
2907 1
2908 1
2909 1
2910 1
2911 1
2912 1
2913 1
2914 1
2915 1
2916 1
2917 1
2918 1
2919 1
2920 1
2921 1
2922 1
2923 1
2924 1
2925 1
2926 1
2927 1
2928 1
2929 1
2930 1
2931 1
2932 1
2933 1
2934 1
2935 1
2936 1
2937 1
2938 1
2939 1
2940 1
2941 1
2942 1
2943 1
2944 1
2945 1
2946 1
2947 1
2948 1
2949 1
2950 1
2951 1
2952 1
2953 1
2954 1
2955 1
2956 1
2957 1
2958 1
2959 1
2960 1
2961 1
2962 1
2963 1
2964 1
2965 1
2966 1
2967 1
2968 1
2969 1
2970 1
2971 1
2972 1
2973 1
2974 1
2975 1
2976 1
2977 1
2978 1
2979 1
2980 1

1406 2
1407 2
1408 2
1409 2
1410 2
1411 2
1412 2
1413 2
1414 2
1415 2
1416 2
1417 2
1418 2
1419 2
1420 2
1421 2
1422 2
1423 2
1424 2
1425 2
1426 2
1427 2
1428 2
1429 2
1430 2
1431 2
1432 2
1433 2
1434 2
1435 2
1436 2
1437 2
1438 2
1439 2
1440 2
1441 2
1442 2
1443 2
1444 2
1445 2
1446 2
1447 2
1448 2
1449 2
1450 2
1451 2
1452 2
1453 2
1454 2
1455 2
1456 2
1457 2
1458 2
1459 2
1460 2
1461 2
1462 2
1463 2
1464 2
1465 2
1466 2
1467 2
1468 2
1469 2
1470 2
1471 2
1472 2
1473 2
1474 2
1475 2
1476 2
1477 2
1478 2
1479 2
1480 2
1481 2
1482 2
1483 2
1484 2
1485 2
1486 2
1487 2
1488 2
1489 2
1490 2
1491 2
1492 2
1493 2
1494 2
1495 2
1496 2
1497 2
1498 2
1499 2
1500 2
1501 2
1502 2
1503 2
1504 2
1505 2
1506 2
1507 2
1508 2
1509 2
1510 2
1511 2
1512 2
1513 2
1514 2
1515 2
1516 2
1517 2
1518 2
1519 2
1520 2
1521 2
1522 2
1523 2
1524 2
1525 2
1526 2
1527 2
1528 2
1529 2
1530 2
1531 2
1532 2
1533 2
1534 2
1535 2
1536 2
1537 2
1538 2
1539 2
1540 2
1541 2
1542 2
1543 2
1544 2
1545 2
1546 2
1547 2
1548 2

2655 2
2656 2
2657 2
2658 2
2659 2
2660 2
2661 2
2662 2
2663 2
2664 2
2665 2
2666 2
2667 2
2668 2
2669 2
2670 2
2671 2
2672 2
2673 2
2674 2
2675 2
2676 2
2677 2
2678 2
2679 2
2680 2
2681 2
2682 2
2683 2
2684 2
2685 2
2686 2
2687 2
2688 2
2689 2
2690 2
2691 2
2692 2
2693 2
2694 2
2695 2
2696 2
2697 2
2698 2
2699 2
2700 2
2701 2
2702 2
2703 2
2704 2
2705 2
2706 2
2707 2
2708 2
2709 2
2710 2
2711 2
2712 2
2713 2
2714 2
2715 2
2716 2
2717 2
2718 2
2719 2
2720 2
2721 2
2722 2
2723 2
2724 2
2725 2
2726 2
2727 2
2728 2
2729 2
2730 2
2731 2
2732 2
2733 2
2734 2
2735 2
2736 2
2737 2
2738 2
2739 2
2740 2
2741 2
2742 2
2743 2
2744 2
2745 2
2746 2
2747 2
2748 2
2749 2
2750 2
2751 2
2752 2
2753 2
2754 2
2755 2
2756 2
2757 2
2758 2
2759 2
2760 2
2761 2
2762 2
2763 2
2764 2
2765 2
2766 2
2767 2
2768 2
2769 2
2770 2
2771 2
2772 2
2773 2
2774 2
2775 2
2776 2
2777 2
2778 2
2779 2
2780 2
2781 2
2782 2
2783 2
2784 2
2785 2
2786 2
2787 2
2788 2
2789 2
2790 2
2791 2
2792 2
2793 2
2794 2
2795 2
2796 2
2797 2

...

2943 8
2944 8
2945 8
2946 8
2947 8
2948 8
2949 8
2950 8
2951 8
2952 8
2953 8
2954 8
2955 8
2956 8
2957 8
2958 8
2959 8
2960 8
2961 8
2962 8
2963 8
2964 8
2965 8
2966 8
2967 8
2968 8
2969 8
2970 8
2971 8
2972 8
2973 8
2974 8
2975 8
2976 8
2977 8
2978 8
2979 8
2980 8
2981 8
2982 8
2983 8
2984 8
2985 8
2986 8
2987 8
2988 8
2989 8
2990 8
2991 8
2992 8
2993 8
2994 8
2995 8
2996 8
2997 8
2998 8
2999 8
3000 8
3001 8
3002 8
3003 8
3004 8
3005 8
3006 8
3007 8
3008 8
3009 8
3010 8
3011 8
3012 8
3013 8
3014 8
3015 8
3016 8
3017 8
3018 8
3019 8
3020 8
3021 8
3022 8
3023 8
3024 8
3025 8
3026 8
3027 8
3028 8
3029 8
3030 8
3031 8
3032 8
3033 8
3034 8
3035 8
3036 8
3037 8
3038 8
3039 8
3040 8
3041 8
3042 8
3043 8
3044 8
3045 8
3046 8
3047 8
3048 8
3049 8
1 9
2 9
3 9
4 9
5 9
6 9
7 9
8 9
9 9
10 9
11 9
12 9
13 9
14 9
15 9
16 9
17 9
18 9
19 9
20 9
21 9
22 9
23 9
24 9
25 9
26 9
27 9
28 9
29 9
30 9
31 9
32 9
33 9
34 9
35 9
36 9
37 9
38 9
39 9
40 9
41 9
42 9
43 9
44 9
45 9
46 9
47 9
48 9
49 9
50 9
51 9
52 9


1241 9
1242 9
1243 9
1244 9
1245 9
1246 9
1247 9
1248 9
1249 9
1250 9
1251 9
1252 9
1253 9
1254 9
1255 9
1256 9
1257 9
1258 9
1259 9
1260 9
1261 9
1262 9
1263 9
1264 9
1265 9
1266 9
1267 9
1268 9
1269 9
1270 9
1271 9
1272 9
1273 9
1274 9
1275 9
1276 9
1277 9
1278 9
1279 9
1280 9
1281 9
1282 9
1283 9
1284 9
1285 9
1286 9
1287 9
1288 9
1289 9
1290 9
1291 9
1292 9
1293 9
1294 9
1295 9
1296 9
1297 9
1298 9
1299 9
1300 9
1301 9
1302 9
1303 9
1304 9
1305 9
1306 9
1307 9
1308 9
1309 9
1310 9
1311 9
1312 9
1313 9
1314 9
1315 9
1316 9
1317 9
1318 9
1319 9
1320 9
1321 9
1322 9
1323 9
1324 9
1325 9
1326 9
1327 9
1328 9
1329 9
1330 9
1331 9
1332 9
1333 9
1334 9
1335 9
1336 9
1337 9
1338 9
1339 9
1340 9
1341 9
1342 9
1343 9
1344 9
1345 9
1346 9
1347 9
1348 9
1349 9
1350 9
1351 9
1352 9
1353 9
1354 9
1355 9
1356 9
1357 9
1358 9
1359 9
1360 9
1361 9
1362 9
1363 9
1364 9
1365 9
1366 9
1367 9
1368 9
1369 9
1370 9
1371 9
1372 9
1373 9
1374 9
1375 9
1376 9
1377 9
1378 9
1379 9
1380 9
1381 9
1382 9
1383 9

2642 9
2643 9
2644 9
2645 9
2646 9
2647 9
2648 9
2649 9
2650 9
2651 9
2652 9
2653 9
2654 9
2655 9
2656 9
2657 9
2658 9
2659 9
2660 9
2661 9
2662 9
2663 9
2664 9
2665 9
2666 9
2667 9
2668 9
2669 9
2670 9
2671 9
2672 9
2673 9
2674 9
2675 9
2676 9
2677 9
2678 9
2679 9
2680 9
2681 9
2682 9
2683 9
2684 9
2685 9
2686 9
2687 9
2688 9
2689 9
2690 9
2691 9
2692 9
2693 9
2694 9
2695 9
2696 9
2697 9
2698 9
2699 9
2700 9
2701 9
2702 9
2703 9
2704 9
2705 9
2706 9
2707 9
2708 9
2709 9
2710 9
2711 9
2712 9
2713 9
2714 9
2715 9
2716 9
2717 9
2718 9
2719 9
2720 9
2721 9
2722 9
2723 9
2724 9
2725 9
2726 9
2727 9
2728 9
2729 9
2730 9
2731 9
2732 9
2733 9
2734 9
2735 9
2736 9
2737 9
2738 9
2739 9
2740 9
2741 9
2742 9
2743 9
2744 9
2745 9
2746 9
2747 9
2748 9
2749 9
2750 9
2751 9
2752 9
2753 9
2754 9
2755 9
2756 9
2757 9
2758 9
2759 9
2760 9
2761 9
2762 9
2763 9
2764 9
2765 9
2766 9
2767 9
2768 9
2769 9
2770 9
2771 9
2772 9
2773 9
2774 9
2775 9
2776 9
2777 9
2778 9
2779 9
2780 9
2781 9
2782 9
2783 9
2784 9

849 10
850 10
851 10
852 10
853 10
854 10
855 10
856 10
857 10
858 10
859 10
860 10
861 10
862 10
863 10
864 10
865 10
866 10
867 10
868 10
869 10
870 10
871 10
872 10
873 10
874 10
875 10
876 10
877 10
878 10
879 10
880 10
881 10
882 10
883 10
884 10
885 10
886 10
887 10
888 10
889 10
890 10
891 10
892 10
893 10
894 10
895 10
896 10
897 10
898 10
899 10
900 10
901 10
902 10
903 10
904 10
905 10
906 10
907 10
908 10
909 10
910 10
911 10
912 10
913 10
914 10
915 10
916 10
917 10
918 10
919 10
920 10
921 10
922 10
923 10
924 10
925 10
926 10
927 10
928 10
929 10
930 10
931 10
932 10
933 10
934 10
935 10
936 10
937 10
938 10
939 10
940 10
941 10
942 10
943 10
944 10
945 10
946 10
947 10
948 10
949 10
950 10
951 10
952 10
953 10
954 10
955 10
956 10
957 10
958 10
959 10
960 10
961 10
962 10
963 10
964 10
965 10
966 10
967 10
968 10
969 10
970 10
971 10
972 10
973 10
974 10
975 10
976 10
977 10
978 10
979 10
980 10
981 10
982 10
983 10
984 10
985 10
986 10
987 10
988 10
989 10
990 10
991 10

2085 10
2086 10
2087 10
2088 10
2089 10
2090 10
2091 10
2092 10
2093 10
2094 10
2095 10
2096 10
2097 10
2098 10
2099 10
2100 10
2101 10
2102 10
2103 10
2104 10
2105 10
2106 10
2107 10
2108 10
2109 10
2110 10
2111 10
2112 10
2113 10
2114 10
2115 10
2116 10
2117 10
2118 10
2119 10
2120 10
2121 10
2122 10
2123 10
2124 10
2125 10
2126 10
2127 10
2128 10
2129 10
2130 10
2131 10
2132 10
2133 10
2134 10
2135 10
2136 10
2137 10
2138 10
2139 10
2140 10
2141 10
2142 10
2143 10
2144 10
2145 10
2146 10
2147 10
2148 10
2149 10
2150 10
2151 10
2152 10
2153 10
2154 10
2155 10
2156 10
2157 10
2158 10
2159 10
2160 10
2161 10
2162 10
2163 10
2164 10
2165 10
2166 10
2167 10
2168 10
2169 10
2170 10
2171 10
2172 10
2173 10
2174 10
2175 10
2176 10
2177 10
2178 10
2179 10
2180 10
2181 10
2182 10
2183 10
2184 10
2185 10
2186 10
2187 10
2188 10
2189 10
2190 10
2191 10
2192 10
2193 10
2194 10
2195 10
2196 10
2197 10
2198 10
2199 10
2200 10
2201 10
2202 10
2203 10
2204 10
2205 10
2206 10
2207 10
2208 10
2209 10
